In [1]:

import math
import wandb
import random
from sequence_replay_buffer import SequenceReplayBuffer
import numpy as np
import pickle as pkl
from collections import deque
from tqdm import tqdm, trange
from typing import Deque, Dict, List, Tuple


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, random_split



## dataset

In [2]:
memory = SequenceReplayBuffer(main_path='/media/ray/intelSSD/mmdemo_train',
                              size = 200,
                              epi_len = 512,
                             )

  0%|          | 0/172 [00:00<?, ?it/s]/home/ray/subt-analyze/08-rl-constrastive/sequence_replay_buffer.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mm_scan'][i] = np.append(df['mm_scan'][i], df['pos_diff'][i])
/home/ray/subt-analyze/08-rl-constrastive/sequence_replay_buffer.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_mm_scan'][i] = np.append(df['next_mm_scan'][i], df['next_pos_diff'][i])
  1%|          | 1/172 [00:00<00:26,  6.36it/s]

/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1411/transition_0717_1411/mmdemo_trans_0717_1411_000.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1411/transition_0717_1411/mmdemo_trans_0717_1411_001.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1411/transition_0717_1411/mmdemo_trans_0717_1411_002.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1411/transition_0717_1411/mmdemo_trans_0717_1411_003.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1411/transition_0717_1411/mmdemo_trans_0717_1411_004.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1426/transition_0717_1426/mmdemo_trans_0717_1426_000.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1426/transition_0717_1426/mmdemo_trans_0717_1426_001.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1426/transition_0717_1426/mmdemo_trans_0717_1426_002.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1454/transition_0717_1454/mmdemo_trans_0717_1454_000.pkl
/media/ray/intelSSD/mmdemo_train/mmdemo_0717_1454/transition_0717_1454/mmdemo_tran

100%|██████████| 172/172 [00:25<00:00,  6.85it/s]


In [3]:
indices = np.random.choice(
                len(memory), size=len(memory), replace=False)


i=0
for sample in indices:
    print(memory[sample]['rews'].shape)
    print(memory[sample]['rews'])
    print(i)
    i+=1


(512,)
[ 2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.4120784
  2.85647    2.8505163  2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.8027968  2.85647
  2.85647    2.631711   2.631711   2.607376   2.85647    2.85647
  2.5956633  2.5427969  2.6367257  2.7865252  2.5962892  2.5882254
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.5962892  2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2

[ 2.81862    2.710271   2.8286345  2.85647    2.85647    2.8385499
  2.5330503  2.5415752  2.4979887  2.4937286  2.5610309  2.5610309
  2.8319507  2.8385499  2.8263743  2.7734199  2.739252   2.7444322
  2.7457182  2.731431   2.731431   2.7444322  2.736652   2.6941016
  2.7182586  2.715603   2.715603   2.7235484  2.7235484  2.6941016
  2.845106   2.8483677  2.8385499  2.85647    2.7072945  2.6953466
  2.631711   2.631711   2.631711   2.631711   2.631711   2.631711
  2.631711   2.631711   2.85647    2.85647    2.85647    2.7457182
  2.7197464  2.7197464  2.85647    2.85647    2.81862    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.787939   2.787939
  2.78793

[ 2.85647    2.85647    2.85647    2.85647    2.85647    2.7196493
  2.818231   2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.7733095  2.7733095  2.7733095  2.7733095  2.7733095
  2.73242    2.6953466  2.6953466  2.6953466  2.6953466  2.6953466
  2.6953466  2.6953466  2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.787939   2.787939
  2.787939   2.787939   2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.85647    2.85647    2.85647    2.85647    2.85647
  2.85647    2.856